In [14]:
import json, urllib

In [177]:
with urllib.request.urlopen("https://raw.githubusercontent.com/hebcal/hebcal-leyning/master/src/aliyot.json") as url:
    aliyot = json.loads(url.read().decode())
with urllib.request.urlopen("https://www.sefaria.org/api/texts/Exodus?vhe=Miqra_according_to_the_Masorah&context=0&pad=0") as url:
    MAPM_ex = json.loads(url.read().decode())    
with urllib.request.urlopen("https://raw.githubusercontent.com/Sefaria/Sefaria-Export/master/json/Tanakh/Torah/Exodus/English/The%20Holy%20Scriptures%20A%20New%20Translation%20JPS%201917.json") as url:
    JPS_ex = json.loads(url.read().decode())
with urllib.request.urlopen("https://raw.githubusercontent.com/Sefaria/Sefaria-Export/master/json/Tanakh/Targum/Onkelos/Torah/Onkelos%20Exodus/Hebrew/Targum%20Onkelos%2C%20vocalized%20according%20to%20the%20Yemenite%20Taj.json") as url:
    targ_ex = json.loads(url.read().decode())
with urllib.request.urlopen("https://raw.githubusercontent.com/Sefaria/Sefaria-Export/master/json/Tanakh/Commentary/Rashi/Torah/Rashi%20on%20Exodus/Hebrew/Rashi%20Chumash%2C%20Metsudah%20Publications%2C%202009.json") as url:
    rashi_ex = json.loads(url.read().decode())

In [187]:
def qkparse(string):
    start = string.index('<span class=\"mam-kq">')
    string_before = string[0:start]
    startk = string.index('<span class=\"mam-kq-k\">',start)
    startk = string.index('>',startk)+2
    endk = string.index('</span>',startk)-1
    ktext = string[startk:endk]
    startq = string.index('<span class=\"mam-kq-q\">')
    startq = string.index('>',startq)+2
    endq = string.index('</span>',startq)-1
    qtext=string[startq:endq]
    end = string.index('</span></span>',endq)
    if " " in string[end:]:
        end = string.index(' ',end)
    else:
        end = string.index('׃',end)
    output = '\\qk{'+qtext+'}{'+ktext+'}'
    string_after = string[end:]
    return string_before+output+string_after

In [272]:
def removeformatting(text):
    text = text.replace("{פ}",r"\petucha ")
    text = text.replace("{ס}",r"\setuma ")
    text = text.replace("<span>","")
    text = text.replace("</span>","")
    text = text.replace("<br>","")
    text = text.replace("<span class=\"mam-kq-trivial\">","")
    text = text.replace("<b>׀</b>",r"\legarmeh ")
    text = text.replace("<small>׀</small>",r"\pasek ")
    text = text.replace("<big>",r"\large ")
    text = text.replace("</big>",r"\normalsize ")
    text = text.replace("<sup>*</sup><i class=\"footnote\">(",r"\note{")
    text = text.replace(')</i>',r"}")
    if "<" in text:
        print(text)
    return text

In [262]:
def parserashi(content, chap, vs):
    rashi_chap = content["text"][chap-1]
    if len(rashi_chap)<vs:
        return ""
    rashi_list = rashi_chap[vs-1]
    rashi_text = ""
    for comment in rashi_list:
        comment = comment.replace("<b>",r"\rashiDH{")
        comment = comment.replace("</b>",r"}")
        comment = comment.replace("<small>","")
        comment = comment.replace("</small>","")
        comment = comment.replace(r"\"","״")
        comment = comment.replace("\"","״")
        comment = comment.replace(r"'","׳")
        comment = comment.replace(r":","׃")
        rashi_text += r"\rashi{"+comment+r"}"
    return rashi_text

In [333]:
def nonalpha_remover(word):
    no_cant_word = ''
    for letter in word:
        if letter.isalpha() == True or letter == "־":
            no_cant_word = no_cant_word + letter
    return no_cant_word

In [217]:
def parsealiyot(aliyot,book_num,vs_id):
    aliyot = dict(reversed(list(aliyot.items())))
    combo = None
    for parsha in aliyot.values():
        if parsha["book"] != book_num:
            continue
        elif parsha["fullkriyah"]["1"]["b"] == vs_id and "combined" not in parsha.keys():
            return ["parsha",nonalpha_remover(parsha["hebrew"])]
        elif parsha["fullkriyah"]["7"]["e"] == vs_id and "combined" not in parsha.keys():
            return ["parsha","end"]
        elif "combined" in parsha.keys():
            aliya_number = 1
            for aliya in parsha["fullkriyah"].values():
                if aliya["b"] == vs_id:
                    combo = aliya_number
                aliya_number += 1
        else:
            aliya_number = 1
            for aliya in parsha["fullkriyah"].values():
                if aliya["b"] == vs_id:
                    if combo == None:
                        return ["aliya",aliya_number]
                    else:
                        return ["combo",[aliya_number, combo]]
                aliya_number += 1
            aliya_number = 1
            if "weekday" in parsha.keys():
                for weekday in parsha["weekday"].values():
                    if weekday["b"]==vs_id:
                        return ["weekday",aliya_number]
                    elif vs_id == parsha["weekday"]["3"]["e"] and vs_id != parsha["fullkriyah"]["1"]["e"]:
                        return ["weekday",4]
                    aliya_number += 1
    return

In [299]:
weekday_aliya_names = [None, None,"לוי","ישראל","ע״כ בחול"]
shabbat_aliya_names = [None,None,"שני","שלישי","רביעי","חמישי","ששי","שביעי","מפטיר"]

In [345]:
def make_chumash(heb, targ, eng, rashi, parsha_data, book):
    versecounter = 1
    chapcounter = 1
    output = []
    for chapter in heb["he"]:
        for verse in chapter:
            id_str = str(chapcounter)+":"+str(versecounter)
            verse_he = verse
            while '<span class=\"mam-kq\">' in verse_he:
                verse_he = qkparse(verse_he)
            verse_he = removeformatting(verse_he)
            heb_id = parsealiyot(parsha_data, 2, id_str)
            if heb_id == None:
                if versecounter == 1:
                    versetext_he = r'\Roman{chap}'
                else:
                    versetext_he = r"\arabic{verse}"
            elif heb_id[0] == "parsha" and heb_id[1]!= "end":
                if chapcounter != 1:
                    #output.append(r"\end{longtable}")
                    output.append(r"}")
                #add end-of-parsha note here
                output.append(r"\newparsha{"+heb_id[1]+"}")
                versetext_he = r"\textsf{"+heb_id[1]+"}"
                output.append(r"\parshabody{")
                if versecounter == 1:
                    versetext_he = r'\Roman{chap}'
                else:
                    versetext_he = r"\arabic{verse}"
            elif heb_id[0] == "weekday":
                aliya_text = weekday_aliya_names[heb_id[1]]
                versetext_he = r"\textsf{"+aliya_text+"}"
            elif heb_id[0] == "combined":
                aliya_text = [shabbat_aliya_names[heb_id[1][1]],shabbat_aliya_names[heb_id[1][0]]]
                versetext_he = r"\textsf{"+aliya_text[0]+r"\newline("+aliya_text[1]+")}"
            elif heb_id[0] == "aliya":
                aliya_text = shabbat_aliya_names[heb_id[1]]
                versetext_he = r"\textsf{"+aliya_text+"}"
            if versecounter == 1:
                versetext_en = r'\Roman{chap}'
            else:
                versetext_en = r"\arabic{verse}"
            rashi_pasuk = parserashi(rashi,chapcounter,versecounter)
            targ_vs = targ["text"][chapcounter-1][versecounter-1]
            english = eng["text"][chapcounter-1][versecounter-1]
            output.append(r'\threeverse{'+versetext_he+"}%Ex."+str(chapcounter)+r':'+str(versecounter))
            if rashi_pasuk != "":
                output.append(r'{'+verse_he)
                output.append(rashi_pasuk+"}")
            else:
                output.append(r'{'+verse_he+'}')
            output.append(r"{"+targ_vs+r"}")
            output.append("{"+english+r"}{"+versetext_en+r"}")
            versecounter += 1
        chapcounter += 1
        output.append(r"\newperek")
        versecounter = 1
    output[-1] = r"}"
    return output

In [346]:
result = make_chumash(MAPM_ex, targ_ex, JPS_ex, rashi_ex, aliyot, 2)

In [347]:
def TeXwriter(outputfile,content):
    with open(outputfile, 'w', encoding='utf-8') as outfile:
        for line in content:
            outfile.write(line)
            outfile.write("\n")

In [348]:
TeXwriter("exodus_multi.tex",result)